In [ ]:
%load_ext autoreload
%autoreload 2
from cppn_torch.activation_functions import *
from matplotlib import pyplot as plt
import imageio.v2 as iio
from tqdm import tqdm
from move import MOVE
from move_config import MoveConfig
from cppn_torch.util import center_crop, resize

In [ ]:
target_name = "https://upload.wikimedia.org/wikipedia/commons/1/1e/Sunrise_over_the_sea.jpg"
# target_name = "https://media.istockphoto.com/id/689364180/vector/smiling-cartoon-face-positive-people-emotion-icon.jpg?s=612x612&w=0&k=20&c=2olrbVI2dkVyR0XKaMs4cSDrzpA4zH4FrCfZ_rKwVjU="


In [ ]:
from run_setup import run_setup, fix_target_dimensions
from evolution_torch.algorithm_config import resize_target
config = MoveConfig()
config.color_mode = "HSL"
config.target =  target_name
config.target_name = 'default'
config.target_res = [100, 100]
pilmode = "RGB" if len(config.color_mode) == 3 else "L"
config.target = torch.tensor(iio.imread(config.target, pilmode=pilmode, as_gray=len(config.color_mode)==1), dtype=torch.float32, device=config.device) / 255.0

import cppn_torch.activation_functions as af
import fitness.fitness_functions as ff
config.activations = [af.identity]
config.objective_functions = [ff.mse, ff.mse, ff.mse, ff.mse]


resize_target(config)
fix_target_dimensions(config)


In [ ]:

plt.imshow(config.target.cpu().numpy())
plt.show()

In [ ]:
move = MOVE(config)

In [ ]:
from cppn_torch.util import show_inputs
show_inputs(move.inputs, cols=10)

In [ ]:
try:
    move.evolve()
except KeyboardInterrupt:
    pass


In [ ]:
from baseline import norm_tensor
normed_fits = torch.ones(move.record.fitness_over_time.shape) * -torch.inf
for fn_i in range(move.record.fitness_over_time.shape[0]):
    fn_name = move.fns[fn_i].__name__
    normed_fits[fn_i] = norm_tensor(move.record.fitness_over_time[fn_i], move.config.norm_df_path, fn_name)

plt.plot(normed_fits.mean(dim=(0)).cpu())
plt.show()

In [ ]:
from util import show_move_map_images
show_move_map_images(move)

In [ ]:
move.solution.draw_nx()

In [ ]:

from cppn_torch.util import visualize_node_outputs
visualize_node_outputs(move.solution, move.inputs);

plt.show()

In [ ]:
print(f"{'Individual in cell' :20s}  {'Lineage'}")
print('-' * 40)
for cell_id, lineage in enumerate(move.get_lineages()):
    print(f"{move.map.cell_names[cell_id]:20s}: {lineage}")
    
print('-' * 40)
print("\n\n* -1: randomly initialized genome")